In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1";

# Importacion de librerias

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import argparse
import tensorflow as tf
from IPython.display import display, clear_output
from random import randint

# Configuraciones de TensorFlow

In [3]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

### Modo eagerly

In [4]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

False

# Carga de Datos

In [5]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

'/home/jefelitman/DataSets/ucf101/split_1'

In [6]:
with tf.compat.v1.device("/CPU:0"):
    batch = 15
    size = [112,112]
    frames = 60
    canales = 3

In [7]:
import sys
sys.path.append('../')

with tf.compat.v1.device("/CPU:0"):
    from Datasets_utils.DatasetsLoader import VideoDataGenerator

    dataset = VideoDataGenerator(directory_path = root_path, 
                                 batch_size = batch, 
                                 frame_size=size, 
                                 video_frames = frames, 
                                 temporal_crop = ('sequential', None), 
                                 frame_crop = (None, None), 
                                 shuffle = True, 
                                 conserve_original = False)

# Red Neuronal

In [8]:
from LTC import LTC

with tf.compat.v1.device("/CPU:0"):
    Sesion = tf.Session(config=config)

    video_shape = [frames] + size + [canales]

    ltc = LTC(101,batch,video_shape)

    ltc.enable_training()

W1031 23:30:36.162958 139642263926592 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:15: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [9]:
with tf.compat.v1.device("/CPU:0"):
    ltc.inicializar_modelo(0.003,0.5)
    init = tf.global_variables_initializer()
    Sesion.run(init)

W1031 23:30:39.337160 139642263926592 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:112: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1031 23:30:39.338214 139642263926592 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:113: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1031 23:30:39.339412 139642263926592 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1031 23:30:39.393835 139642263926592 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:157: The name tf.nn.xw_plus_b is deprecated. Please use tf.compat.v1.nn.xw_plus_b instea

In [10]:
#Como es primera ver corriendo este modelo entonces no debo cargar nada
saver = tf.train.Saver()
step = 1
train_epoch = 1
test_epoch = 1
train_loss = []
train_accuracy = []
test_loss = []
test_accuracy = []
while True:
    #Primero voy salvando el modelo de 40000 en 40000
    if step % 1000 == 0:
        saved_path = saver.save(Sesion, '../Checkpoints/LTC_112x112x60_rgb_batch15/LTC_112x112x60_rgb')
        print('Modelo salvado en: {}'.format(saved_path))
        
    if step == 80000 or step == 125000:
        ltc.lr = ltc.lr * 0.1
    
    if step % 10 == 0:
        clear_output(wait=True)
        print(" Step: ",step)
        print("Train Epoch: ",train_epoch, " Train batch: ",dataset.train_batch_index+1,"/",dataset.train_batches)
        print("Train_Loss: ",train_loss[-1]," Train_Acuraccy: ",train_accuracy[-1])
        print("Test Epoch: ",test_epoch," Test batch: ",dataset.test_batch_index+1,"/",dataset.test_batches)
        print("Test_Loss: ",test_loss[-1]," Test_Acuraccy: ",test_accuracy[-1])
    
    if dataset.train_batch_index == dataset.train_batches:
        train_epoch += 1
        
    if dataset.test_batch_index == dataset.test_batches:
        test_epoch += 1
    
    #Entreno con datos de entrenamiento
    batch, labels = dataset.get_next_train_batch(canales)
    Sesion.run(ltc.entrenar,feed_dict={ltc.x: batch, ltc.y: labels})
    train_loss.append(Sesion.run(ltc.perdida,feed_dict={ltc.x: batch, ltc.y: labels}))
    train_accuracy.append(Sesion.run(ltc.precision,feed_dict={ltc.x: batch, ltc.y: labels}))
    
    #Pruebo con datos de test
    batch, labels = dataset.get_next_test_batch(canales)
    test_loss.append(Sesion.run(ltc.perdida,feed_dict={ltc.x: batch, ltc.y: labels}))
    test_accuracy.append(Sesion.run(ltc.precision,feed_dict={ltc.x: batch, ltc.y: labels}))
    
    step += 1

 Step:  4600
Train Epoch:  3  Train batch:  1190 / 1705
Train_Loss:  831.69366  Train_Acuraccy:  0.0
Test Epoch:  7  Test batch:  604 / 666
Test_Loss:  799.4576  Test_Acuraccy:  0.0


KeyboardInterrupt: 

In [11]:
max(train_accuracy)

0.2

In [14]:
max(test_accuracy)

0.13333334